# 1. Import data

In [64]:
import itertools
import pandas as pd
import numpy as np 
from datetime import datetime, timedelta
import networkx as nx
import matplotlib.pyplot as plt

# some global constants
_FORMAT_TIME = '%H:%M'
_LARGEST_DELAY = 15 # in minutes, largest delay of vehicles considered at each station
_DEFAULT_YEAR_ = 2018
_MAX_STATIONS_ = 8
_MIN_PROB_THRES_  = 0.5
_WALKING_VELOCITY = 1.0
_WALKING_VARIANCE = 0.2
_RELAXATION_FACTOR= -20

We will focus on all the stops within 10km of the Zurich train station.

We calculated distance using this formula:

$$ DISTANCE = 2* arcsin\sqrt{sin^2\frac{a}{2}+cos(Lat1)*cos(Lat2)*sin^2\frac{b}{2}} * Earth Radius $$
$$ a = Lat1-Lat2, b = Lon1 - Lon2 $$

In [2]:
geo = pd.read_csv('./data/BFKOORD_GEO', sep="%", header=None,error_bad_lines=False)
geo.columns = ['data','name']
geo.name = geo.name.apply(str.lstrip).apply(str.rstrip)

geo[['station_number','longtitude','latitude','height']] = geo.data.str.split(expand=True)#.apply(float)
geo.drop('data',axis=1,inplace=True)

# station in Zurich
zurich = geo[geo.station_number=="8503000"].reset_index(drop=True)

In [3]:
zurich

,name,station_number,longtitude,latitude,height
0,Zürich HB,8503000,8.540192,47.378177,408


In [4]:
# Define a function to compute the distance with the longtitude and the latitude
from math import sin, cos, sqrt, atan2, radians,asin
def compute_distance(point_1_lat, point_1_lon, point_2_lat=zurich.latitude, point_2_lon=zurich.longtitude):
    # approximate radius of earth in km
    R = 6378.137 # earth radius

    lat1 = radians(float(point_1_lat))
    lon1 = radians(float(point_1_lon))
    lat2 = radians(float(point_2_lat))
    lon2 = radians(float(point_2_lon))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))

    distance = R * c
    return np.round(distance,3) # return distance in kilometres

In [5]:
distance = []
for log,lat in zip(geo.longtitude,geo.latitude):
    distance.append(compute_distance(lat,log))

## find the stations lie inside the radius of 10km
geo['distance'] = distance
zurich_neigh_station = geo[geo.distance <= 10]

# 2. Process the dataframe 

We build seperate graphs for weekday, weekend and National Holiday, because different schedules are applied to them. In this project, we choose 28.04.2018, 29.04.2018, 30.04.2018 to build our graph, which can be changed in the argument "date".

Note that we assume the schedule for weekday (weekend, National Holiday) does not change throughout the year. so we can pick one day to construct the graph.

In [6]:
date = '30.04.2018'
df = pd.read_csv("./data/grouped_201804.csv")
df_day = df[(df['date_of_trip'] == date)&(df['additional_trip'] == False)&(df['not_stop'] == False)]
df_day = df_day.reset_index(drop=True)
nan = np.nan
df_day['Timetable'] = df_day.Timetable.map(lambda x:eval(x))
df_day = df_day[['date_of_trip', 'identifies_of_trip', 'Timetable', 'train_number']]
all_station = []
for idx, row in df_day.iterrows():
    station = []
    for i in range(len(row['Timetable'])):
        station.append(row['Timetable'][i][0])
    all_station.append(station)
df_day['station_name'] = all_station

In [7]:
df_day.head(10)

,date_of_trip,identifies_of_trip,Timetable,train_number,station_name
0,30.04.2018,85:11:1507:002,"[[Zürich HB, 23400, 23593, 23940, 23984], [Zür...",1507,"[Zürich HB, Zürich Flughafen]"
1,30.04.2018,85:11:1509:003,"[[Zürich HB, 27000, 27113, 27540, 27582], [Zür...",1509,"[Zürich HB, Zürich Flughafen]"
2,30.04.2018,85:11:1510:003,"[[Zürich Flughafen, 25860, 25906, 25980, 26102...",1510,"[Zürich Flughafen, Zürich HB]"
3,30.04.2018,85:11:1511:003,"[[Zürich HB, 30600, 30690, 31140, 31223], [Zür...",1511,"[Zürich HB, Zürich Flughafen]"
4,30.04.2018,85:11:1512:003,"[[Zürich Flughafen, 29460, 29451, 29580, 29710...",1512,"[Zürich Flughafen, Zürich HB]"
5,30.04.2018,85:11:1513:003,"[[Zürich HB, 34200, 34329, 34740, 34776], [Zür...",1513,"[Zürich HB, Zürich Flughafen]"
6,30.04.2018,85:11:1514:003,"[[Zürich Flughafen, 33060, 33121, 33180, 33300...",1514,"[Zürich Flughafen, Zürich HB]"
7,30.04.2018,85:11:1515:003,"[[Zürich HB, 37800, 37948, 38340, 38411], [Zür...",1515,"[Zürich HB, Zürich Flughafen]"
8,30.04.2018,85:11:1516:001,"[[Zürich Flughafen, 36660, 36612, 36780, 36838...",1516,"[Zürich Flughafen, Zürich HB]"
9,30.04.2018,85:11:1517:001,"[[Zürich HB, 41400, 41402, 41940, 41988], [Zür...",1517,"[Zürich HB, Zürich Flughafen]"


In [8]:
#generate lineid-tripid dictionary.
lineid_trip = df_day[["train_number", "identifies_of_trip"]] \
            .groupby(["train_number"])["identifies_of_trip"] \
            .apply(lambda x: "%s" % ','.join(x))
lineid_trip = lineid_trip.to_frame().reset_index()
lineid_trip['identifies_of_trip'] = lineid_trip['identifies_of_trip'].map(lambda x: list(set(x.split(','))))
lineid_trip_dict = dict(zip(lineid_trip.train_number, lineid_trip.identifies_of_trip))

In [9]:
#generate tripid-station dictionary
trip_id_dict = dict()
for _, row in df_day.iterrows():
    trip_id = row['identifies_of_trip']
    one_trip = dict()
    for item in row['Timetable']:
        arr_time = item[1]
        arr_time = '{}:{}'.format(arr_time//3600, (arr_time%3600)//60)
        dep_time = item[3]
        dep_time = '{}:{}'.format(dep_time//3600, (dep_time%3600)//60)
        one_trip[item[0]] = [arr_time, dep_time]
    trip_id_dict[trip_id] = one_trip

In [10]:
import pickle
with open("./data/timetable180430_test.pickle", 'wb') as handle:
    pickle.dump(trip_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open("./data/lineid_trip180430_test.pickle", 'wb') as handle:
    pickle.dump(lineid_trip_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 3. Add neighbor station 

We also add "Walking" as one specific transportation type in the graph. Here, we calculate the minimum/maximum latitude/longtitude range of a fixed station within a fixed distance.

In [11]:
## calculate minimum/maximum latitude/longtitude given a fixed distance
def compute_distance_inverse(lat, lng, distance):
    radius = 6371

    ## latitude boundaries
    maxlat = lat + np.rad2deg(distance / radius)
    minlat = lat - np.rad2deg(distance / radius)

    ## longitude boundaries (longitude gets smaller when latitude increases)
    maxlng = lng + np.rad2deg(distance / radius / np.cos(np.deg2rad(lat)))
    minlng = lng - np.rad2deg(distance / radius / np.cos(np.deg2rad(lat)))
    return maxlat, minlat, maxlng, minlng

## add neighbor station for each station
zurich_neigh_station = zurich_neigh_station.reset_index()
zurich_neigh_station = zurich_neigh_station.drop(columns = ["index"])
zurich_neigh_station.longtitude = zurich_neigh_station.longtitude.apply(lambda x: float(x))
zurich_neigh_station.latitude = zurich_neigh_station.latitude.apply(lambda x: float(x))

lng = zurich_neigh_station.longtitude.values
lat = zurich_neigh_station.latitude.values
maxlat, minlat, maxlng, minlng = compute_distance_inverse(lat, lng, distance = 0.2)

neighbor = []

for i in range (zurich_neigh_station.shape[0]):
    loc = np.where(((maxlat[i] > lat) == True) & ((minlat[i] < lat) == True) & ((maxlng[i] > lng) == True) & ((minlng[i] < lng) == True))
    neighbor.append(((zurich_neigh_station.name.values[loc[0]])))
d = pd.Series(neighbor)
zurich_neigh_station['neighbor'] = d
zurich_neigh_station.head(10)

,name,station_number,longtitude,latitude,height,distance,neighbor
0,Zimmerberg-Basistunnel,0000176,8.521961,47.351679,0,3.254,[Zimmerberg-Basistunnel]
1,Urdorf,8502220,8.434713,47.390882,442,8.075,"[Urdorf, Urdorf, Bahnhof]"
2,Birmensdorf ZH,8502221,8.437543,47.357432,488,8.076,"[Birmensdorf ZH, Birmensdorf ZH, Bahnhof, Birm..."
3,Bonstetten-Wettswil,8502222,8.468175,47.325896,528,7.961,"[Bonstetten-Wettswil, Bonstetten-Wettswil, Bah..."
4,Urdorf Weihermatt,8502229,8.430330,47.380971,456,8.287,"[Urdorf Weihermatt, Urdorf Weihermatt, Bahnhof]"
5,"Waldegg, Birmensdorferstrasse",8502559,8.463472,47.368305,588,5.887,"[Waldegg, Birmensdorferstrasse, Waldegg, Post]"
6,"Zürich, Goldbrunnenplatz",8502572,8.513918,47.370293,421,2.166,"[Zürich, Goldbrunnenplatz, Zürich, Zwinglihaus]"
7,"Aesch ZH, Gemeindehaus",8502876,8.438705,47.338209,537,8.852,"[Aesch ZH, Gemeindehaus]"
8,"Bonstetten, Dorfplatz",8502885,8.467781,47.315088,528,8.897,"[Bonstetten, Dorfplatz]"
9,"Birmensdorf ZH, Zentrum",8502950,8.437173,47.353936,468,8.223,"[Birmensdorf ZH, Zentrum]"


# 4. Build the graph

We make use of the networkx package to build the graph. Firstly, we add all the edges in the multi-graph. Secondly, we add the walking as an additional transportation type. Then we reduce the multi-edge graph to single edge graph, which reduces the graph searching time.

In [12]:
def BuildGraph():
    
    ## Build the Multidigraph to record all the paths
    Multi_G = nx.MultiDiGraph()
    for index, item in df_day.iterrows():
        train_number = item["train_number"]
        num_stop = len(item["station_name"])
        for i in range (num_stop - 1):
            arrival = item["station_name"][i + 1]
            departure = item["station_name"][i]
            Multi_G.add_edge(departure, arrival, train_number = train_number)

    ## Add the walking edge in multidigraph
    cnt = 0
    nodes = list(Multi_G.nodes)
    for index, item in zurich_neigh_station.iterrows():
        departure = item["name"]
        for i in range(len(item["neighbor"])):
            ## give walking edge an unique id
            cnt += 1
            if(item["neighbor"].size != 0):
                arrival = item["neighbor"][i]
                if((departure in nodes) & (arrival in nodes)):
                    placeAdf = zurich_neigh_station[zurich_neigh_station['name']==arrival]
                    placeBdf = zurich_neigh_station[zurich_neigh_station['name']==departure]
                    distance = compute_distance(placeAdf.latitude.values[0], 
                                placeAdf.longtitude.values[0], 
                                placeBdf.latitude.values[0], 
                                placeBdf.longtitude.values[0])
                    Multi_G.add_edge(departure, arrival, train_number = "Walk_" + str(cnt) + "," + str(distance))
    
    ## Reduce Multidigraph to digraph
    G = nx.DiGraph()
    nodes = list(Multi_G.nodes)
    for i in range (len(nodes)):
        for j in range(len(nodes)):    
            stop_1 = nodes[i]
            stop_2 = nodes[j]        
            if (i != j):
                edge_dict = Multi_G.get_edge_data(stop_1, stop_2)
                path_list = set()      
                ## transform multiedge into one edge with a list of labels
                if(edge_dict != None):
                    for k in edge_dict.keys():
                        path_list.add(edge_dict[k]['train_number'])
                    G.add_edge(stop_1, stop_2, train_number = path_list)
    return G

In [19]:
def find_path(G, start, destination, max_edges):
    '''
        It searches all possible paths on the given graph with the given starts,
        destinations and max_edges.
    '''
    trips = []
    for path in nx.all_simple_paths(G, source = start, target = destination, cutoff = max_edges):
        vehicles = []
        for i in range (len(path) - 1):
            stop_1 = path[i]
            stop_2 = path[i + 1]
            edge_dict = G.get_edge_data(stop_1, stop_2)
            vehicles.append(list(edge_dict['train_number']))
        trips.append([path, vehicles])
    return trips   

In [14]:
G = BuildGraph()

In [15]:
nx.write_gpickle(G, "./data/graph_200_0430.gpickle")

In [46]:
trip = find_path(G, start = 'Zürich Oerlikon', destination = 'Opfikon, Bahnhof', max_edges = 3)
print('One route:\n', trip[0][0])
print('The corresponding bus/train number:\n', trip[0][1])

One route:
 ['Zürich Oerlikon', 'Glattbrugg', 'Glattbrugg, Bahnhof', 'Opfikon, Bahnhof']
The corresponding bus/train number:
 [['18990', '19528', '19560', '19518', '19574', '18934', '18976', '18952', '19572', '18950', '18068', '18060', '19556', '19564', '19554', '18924', '18964', '19580', '19534', '18960', '18920', '18986', '18984', '18928', '19532', '18982', '18954', '19540', '19546', '18932', '18968', '19570', '19588', '19536', '19582', '19544', '19584', '19578', '19548', '18958', '18926', '18946', '19524', '19538', '18936', '18938', '19530', '18942', '19568', '18064', '19552', '19550', '19562', '18970', '18980', '19586', '18966', '19576', '19520', '18972', '18944', '19542', '18940', '19526', '19590', '19558', '18930', '19566', '19122', '18992', '18988', '18948', '18974', '18916', '18922', '18978', '19522', '18956', '18962'], ['Walk_162,0.049'], ['85:773:761', 'Walk_791,0.204']]


# 5. Route filtering
## 5.1 aggregate trips
As shown above, each station corresponds to many trips (represented by bus/train number). However, we prefer to find direct trips and those trips having few interchanges. Therefore, we have to aggregate these trips. 

In [30]:
def aggregate_trips(trips_candidates: list, num_interchange: int = 10):
    '''
        It keeps all direct trips and those trips having few interchanges. 
        Firstly, we convert all trips into set, then we conduct intersection 
        on these sets to figure out all transfers.
    '''
    candidates_trips = []
    for trip in trips_candidates:
        
        # load bus and station 
        bus_list = trip[1]
        station_list = trip[0]
        
        # initialize set
        new_set = {}
        old_set = {}
        aggre_trip = []
        aggre_statation = [station_list[0]]
        
        cnt = 0
        while cnt <= len(bus_list):
            if len(new_set) == 0:
            # new_set being empty meanings this is the first set or this station 
            # is a transfer.
                if len(old_set) != 0:
                # old_set not empty means this is a transfer
                    cnt -= 1
                    aggre_trip.append(list(old_set))
                    aggre_statation.append(station_list[cnt])
                    
                    new_set = set(bus_list[cnt])
                    old_set = {}
                else:
                    new_set = set(bus_list[cnt])
            else:
                # sets intersection to find a path
                old_set = new_set
                if cnt < len(bus_list):
                    new_set = new_set.intersection(set(bus_list[cnt]))   
            cnt += 1
 
        # append possible trips (might be unresonable)
        aggre_trip.append(list(new_set))
        aggre_statation.append(station_list[-1])
        
        if len(aggre_trip) <= num_interchange:
        # throw trips with more than num_interchange transfer
            candidates_trips.append([aggre_statation, aggre_trip])
    
    return candidates_trips

## 5.2 filter trips with time constraints
Thought trips have been aggregated, as there is no time constraint such as arrival time and departure time, the 'aggreagate_trips' might contain some unreasonable trips. For exmaple, a bus departs at 14:30 while the user wants to departure at 12:30.

Thus, a filter based on time constraints is implemented. 

In [48]:
def filter_trips(trips_list: list, bus_to_tripId: dict, time_table: dict, dist_map: pd.DataFrame, start_time: str):
    '''
        It filters out trips that are unresonable. In order to reduce the combination, 
        the filer is separated into two phases:
            1. roughly filtering: filter trips by group to reduce the number of combination
            2. second filtering : filter trips by path
    '''
    # First phase
    trips_list = time_filter(trips_list, bus_to_tripId, time_table, dist_map, start_time )
    
    # Second phase
    candidates_trips = []
    for trip in trips_list:
        new_paths = []
        buses = trip[1]
        stops = trip[0]
        # throw invalid trips
        if buses is None:
            continue
        
        trip_combination = [ i for i in itertools.product(*buses)]
        stops_list = [stops] * len(trip_combination)
        for ind in range(len(trip_combination)):
            new_paths.append( [ stops_list[ind], [ [bus] for bus in trip_combination[ind]] ])
       
        path_list = time_filter(new_paths, bus_to_tripId, time_table, dist_map, start_time, second_phase=True)
        candidates_trips.append(path_list)
        
    return candidates_trips



def time_filter(trips_list: list, bus_to_tripId: dict, time_table: dict, dist_map: pd.DataFrame, start_time: str, second_phase:bool=False):
    '''
        It uses time constraint to filter trips
    '''
    for ind_trips in range(len(trips_list)):
        
        stations, vehicle = trips_list[ind_trips][0], trips_list[ind_trips][1]
        
        old_trips = None
        walk_arr_early = None
        walk_arr_late  = None
        
        for ind in range(len(vehicle)):
        # for the ind-th stop    
            new_transfer = stations[ind]
            new_buses = vehicle[ind]
            
            if old_trips is None:
            # if this is the first stop then use the given start time 
                start_time_earliest = datetime.strptime(start_time, _FORMAT_TIME)
                start_time_latest = start_time_earliest
                
            else:    
            # else this is an intermediate stop then use the latest arrival time as the start time 
                bus_arr_time = [datetime.strptime(time_table[trip][new_transfer][0], _FORMAT_TIME) for trip in old_trips if 'Walk' not in trip]
                    
                if 'Walk' in old_trips:
                    bus_arr_time += [walk_arr_early, walk_arr_late]
                
                arr_time = sorted(bus_arr_time)
                start_time_earliest = arr_time[0]
                start_time_latest = arr_time[-1]
            
            next_transfer = stations[ind+1]
            if sum(['Walk' in choice for choice in new_buses]):
            # if new trips contain walk
                dist = dist_map[ (dist_map['placeA'] == new_transfer) & (dist_map['placeB'] == next_transfer)].distance.values*1000
                walk_arr_early= timedelta(seconds = dist[0]/_WALKING_VELOCITY) + start_time_earliest
                walk_arr_late = timedelta(seconds = dist[0]/_WALKING_VELOCITY) + start_time_latest
                
            filtered_tripId = time_constraint(new_transfer, new_buses, next_transfer, bus_to_tripId, time_table, 
                                              start_time_earliest, start_time_latest, enable_second_phase=second_phase)
            
            if len(filtered_tripId) == 0:
            # if one of the intermediate trips is invalid, throw all current trips
                trips_list[ind_trips][1] = None
                break 
            
            old_trips = filtered_tripId
            trips_list[ind_trips][1][ind] = filtered_tripId
            
    return trips_list


def time_constraint(new_transfer : str, new_buses: list, next_transfer: str, bus_to_tripId: dict, 
                    time_table: dict, start_time_early: str, start_time_late: str, enable_second_phase: bool):
    '''
        It uses two simple time constraints to filter unresonable trips:
            1. departure time must be later than arrival time.
            2. departure time must be earier than arrival time + _LARGEST_DELAY.
    '''
    candidates = []
    for bus in new_buses:
        
        if 'Walk' in bus:
            trip_list = ['Walk']
        elif enable_second_phase:
            trip_list = new_buses               # for second phase filtering
        else:
            trip_list = bus_to_tripId[bus]      # if is not walk, load tripId
             
        for trip in trip_list:            
            if trip == 'Walk':
                candidates.append(trip)
            else:
                try:
                    # if the trip does not stop at the station, we throw the trip.
                    # This is possible when some trains share the same train_number while 
                    # they have different trips and one the target_arr trip pass the station.
                    dep_time = datetime.strptime(time_table[trip][new_transfer][1], _FORMAT_TIME)
                    if time_table[trip][next_transfer][0] == 'nan:nan':
                        continue
                    elif datetime.strptime(time_table[trip][next_transfer][0], _FORMAT_TIME) < dep_time:
                        continue
                except:
                    continue
                    
                if dep_time is None:
                    # if the time is invalid (NaN), we throw this trip
                    continue
                else:
                    # compute time difference
                    time_diff_early = dep_time - start_time_early
                    time_diff_late  = dep_time - start_time_late
                    minutes_late = int(time_diff_late.total_seconds()/60)
                   
                    if minutes_late >= _LARGEST_DELAY or int(time_diff_early.total_seconds()) <= _RELAXATION_FACTOR:
                    # throw trips being later than _LARGEST_DELAY and earler than the earliest trip
                        continue
                    else:
                        candidates.append(trip)
            
    return candidates

In [52]:
# load data according to the given date, as trips are 
# different for weekday, weekend and pubilc holiday.

def load_data(date: datetime):
    
    weekday = date.weekday()
    
    if  weekday <= 4:   # weekday data
        path_schedual = './data/schedule_430.csv'  
        path_graph    = './data/graph_200_0430.gpickle'  
        path_lineID   = './data/lineid_trip180430.pickle'   
        path_stdtime  = './data/timetable180430.pickle'   
        
    elif weekday == 5:  # weekend data
        path_schedual = './data/schedule_428.csv'        
        path_graph    = './data/graph_200_0428.gpickle'
        path_lineID   = './data/lineid_trip180428.pickle'
        path_stdtime  = './data/timetable180428.pickle'
        
    else:               # public holiday data
        path_schedual = './data/schedule_429.csv'        
        path_graph    = './data/graph_200_0429.gpickle'    
        path_lineID   = './data/lineid_trip180429.pickle'       
        path_stdtime  = './data/timetable180429.pickle'    
        
    with open(path_lineID,'rb') as file:       # bus number -> tripID
        line_id = pickle.load(file)                             
    
    with open(path_stdtime,'rb') as file:      # tripID -> time
        tripID_2_time = pickle.load(file)
        
    time_table = pd.read_csv(path_schedual)                             # table to get the absolute time of a given (tripsId, station_name) pair
    graph = nx.read_gpickle(path_graph)                                 # graph
    dist_map = pd.read_csv('./data/distMap.csv')                        # distance map betew
    trip_table = pd.read_csv('./data/group_dense_time_table.csv')       # tripID -> all available time difference
    
    return time_table, graph, dist_map, trip_table, tripID_2_time, line_id, weekday

In [60]:
# set some data for demostration 
departure_month = 'Apr'
departure_day   = 30
departure_hour  = 12
departure_minute= 30

arrival_month = 'Apr'
arrival_day   = 30
arrival_hour  = 13
arrival_minute= 30

# convert format
dep_date = datetime.strptime( departure_month+ ' {:02d} {:02d}'.format(departure_day, _DEFAULT_YEAR_), '%b %d %Y')
dep_time = '{:02d}:{:02d}'.format(departure_hour, departure_minute)

arr_date = datetime.strptime( arrival_month + ' {:02d} {:02d}'.format(arrival_day, _DEFAULT_YEAR_)   , '%b %d %Y')
arr_time = '{:02d}:{:02d}'.format(arrival_hour, arrival_minute)

# load necessary tables
date = datetime.strptime( departure_month+ ' {:02d} {:02d}'.format(departure_day, _DEFAULT_YEAR_), '%b %d %Y')
time_table, graph, dist_map, trip_table, tripID_2_time, line_id, weekday = load_data(date)

In [65]:
# aggregating trips
aggregate_trip = aggregate_trips(trips_candidates=trip, num_interchange=3)

# roughly filtering trips
filtered_trips = filter_trips(aggregate_trip, line_id, tripID_2_time, dist_map, dep_time)

In [76]:
for ind, trip in enumerate(filtered_trips):
    print('Route {:d}: {:s}'.format(ind, str(trip[0][0])))
    print('The corresponding bus/train number:', trip[0][1])
    print('\n')

Route 0: ['Zürich Oerlikon', 'Glattbrugg', 'Glattbrugg, Bahnhof', 'Opfikon, Bahnhof']
The corresponding bus/train number: [['85:11:19544:001'], ['Walk'], ['Walk']]


Route 1: ['Zürich Oerlikon', 'Glattbrugg', 'Glattbrugg, Post', 'Opfikon, Bahnhof']
The corresponding bus/train number: [['85:11:19544:001'], ['Walk'], ['Walk']]




---